In [1]:
import pandas as pd
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score,precision_score,f1_score

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [3]:
from google.colab import drive
drive.mount('/gdrive')
df = pd.read_excel('/gdrive/MyDrive/Sentiment_Dataset.xlsx')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


FileNotFoundError: ignored

In [4]:
df.info()

NameError: ignored

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('english'))
class CustomTokenizer:
    def tokenize(self, text):
        tokens = nltk.word_tokenize(text)
        lowercased_tokens = [token.lower() for token in tokens]
        filtered_tokens = [token for token in lowercased_tokens if token not in stop_words]
        return filtered_tokens



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   2745 non-null   string
 1   Sentiment  2748 non-null   int64 
dtypes: int64(1), string(1)
memory usage: 43.1 KB


In [ ]:
df['Sentence']=df['Sentence'].astype('string')

In [ ]:
df.head()

,Sentence,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
# Create an instance of the custom tokenizer
tokenizer = CustomTokenizer()

# Tokenize and preprocess the text column
processed_text = []
for text,tx in df.iterrows():
  ok=str(tx.Sentence)
  processed_text.append(' '.join(tokenizer.tokenize(ok)))

In [ ]:
dfc=df.copy()

In [ ]:
dfc['Sentence']=processed_text

In [ ]:
dfc.head()

,Sentence,Sentiment
0,wow ... loved place .,1
1,crust good .,0
2,tasty texture nasty .,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices .,1
